In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@2600fd33

In [2]:
import spark.implicits._
import java.sql.Date

case class Activity(
  user_id: Int,
  session_id: Int,
  activity_date: Date,
  activity_type: String
)

val activityData: Seq[(Int, Int, Date, String)] = Seq(
  (1, 1, Date.valueOf("2019-07-20"), "open_session"),
  (1, 1, Date.valueOf("2019-07-20"), "scroll_down"),
  (1, 1, Date.valueOf("2019-07-20"), "end_session"),
  (2, 4, Date.valueOf("2019-07-20"), "open_session"),
  (2, 4, Date.valueOf("2019-07-21"), "send_message"),
  (2, 4, Date.valueOf("2019-07-21"), "end_session"),
  (3, 2, Date.valueOf("2019-07-21"), "open_session"),
  (3, 2, Date.valueOf("2019-07-21"), "send_message"),
  (3, 2, Date.valueOf("2019-07-21"), "end_session"),
  (4, 3, Date.valueOf("2019-06-25"), "open_session"),
  (4, 3, Date.valueOf("2019-06-25"), "end_session")
)

val activityDS = activityData.toDF(
  "user_id",
  "session_id",
  "activity_date",
  "activity_type"
).as[Activity]


import spark.implicits._

import java.sql.Date


defined class Activity
activityData: Seq[(Int, Int, Date, String)] = List(
  (1, 1, 2019-07-20, "open_session"),
  (1, 1, 2019-07-20, "scroll_down"),
  (1, 1, 2019-07-20, "end_session"),
  (2, 4, 2019-07-20, "open_session"),
  (2, 4, 2019-07-21, "send_message"),
  (2, 4, 2019-07-21, "end_session"),
  (3, 2, 2019-07-21, "open_session"),
  (3, 2, 2019-07-21, "send_message"),
  (3, 2, 2019-07-21, "end_session"),
  (4, 3, 2019-06-25, "open_session"),
  (4, 3, 2019-06-25, "end_session")
)
activityDS: Dataset[Activity] = [user_id: int, session_id: int ... 2 more fields]

Write a solution to find the daily active user count for a period of 30 days ending 2019-07-27 inclusively. A user was active on someday if they made at least one activity on that day.

Return the result table in any order.

The result format is in the following example.

| day        | active_users |
|------------|--------------| 
| 2019-07-20 | 2            |
| 2019-07-21 | 2            |

In [3]:
activityDS.show()
activityDS.filter(
        (col("activity_date") > date_add(to_date(lit("2019-07-27")), -29)) &&
        (col("activity_date") <= to_date(lit("2019-07-27")))
    )
    .groupBy(col("activity_date"))
    .agg(count_distinct(col("user_id")).as("active_users"))
    .show()

+-------+----------+-------------+-------------+
|user_id|session_id|activity_date|activity_type|
+-------+----------+-------------+-------------+
|      1|         1|   2019-07-20| open_session|
|      1|         1|   2019-07-20|  scroll_down|
|      1|         1|   2019-07-20|  end_session|
|      2|         4|   2019-07-20| open_session|
|      2|         4|   2019-07-21| send_message|
|      2|         4|   2019-07-21|  end_session|
|      3|         2|   2019-07-21| open_session|
|      3|         2|   2019-07-21| send_message|
|      3|         2|   2019-07-21|  end_session|
|      4|         3|   2019-06-25| open_session|
|      4|         3|   2019-06-25|  end_session|
+-------+----------+-------------+-------------+

+-------------+------------+
|activity_date|active_users|
+-------------+------------+
|   2019-07-21|           2|
|   2019-07-20|           2|
+-------------+------------+



In [4]:
spark.stop()